In [1]:
# --- BƯỚC 1: KẾT NỐI & CÀI ĐẶT ---
from google.colab import drive
import os
import torch

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Cài đặt các thư viện cần thiết cho YOLOv5 (nếu chưa có)
# Chúng ta cài trực tiếp từ repo của ultralytics để đảm bảo tương thích
!pip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt
!pip install opencv-python-headless

print("✅ Đã kết nối Drive và cài đặt môi trường xong!")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.1 MB/s eta 0:00:00
✅ Đã kết nối Drive và cài đặt môi trường xong!


In [3]:
# --- BƯỚC 2: CHẠY NHẬN DIỆN & LỌC NGƯỜI ĐI BỘ ---
import cv2
import torch
import numpy as np
from IPython.display import clear_output

# ================= CẤU HÌNH (BẠN CHỈ CẦN SỬA Ở ĐÂY) =================
# 1. Đường dẫn đến file Model của bạn trên Drive
# (Tìm file best.pt trong folder runs/train/exp... nào tốt nhất)
MY_MODEL_PATH = '/content/drive/MyDrive/yolov5/runs/train/exp2/weights/best.pt'

# 2. Đường dẫn file VIDEO hoặc ẢNH đầu vào (Upload lên Colab rồi copy path)
INPUT_PATH = '/content/xeco.mp4'

# 3. Tên file kết quả muốn lưu
OUTPUT_PATH = '/content/ket_qua_da_loc.mp4'
# ====================================================================

def run_smart_detection():
    print(f"⏳ Đang tải model từ: {MY_MODEL_PATH} ...")

    # Load Model Xe máy (Có sẵn trên mạng)
    try:
        model_vehicle = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True, verbose=False)
        model_vehicle.classes = [3]  # Class 3 = motorcycle
        model_vehicle.conf = 0.4
    except Exception as e:
        print(f"Lỗi tải model xe máy: {e}")
        return

    # Load Model Mũ bảo hiểm (Của bạn)
    try:
        model_helmet = torch.hub.load('ultralytics/yolov5', 'custom', path=MY_MODEL_PATH, verbose=False)
        model_helmet.conf = 0.5
    except Exception as e:
        print(f"❌ LỖI: Không tìm thấy model của bạn tại {MY_MODEL_PATH}")
        print("👉 Hãy kiểm tra lại đường dẫn file best.pt trên Drive.")
        return

    print("🚀 Đang xử lý video... (Sẽ mất một lúc tùy độ dài)")

    cap = cv2.VideoCapture(INPUT_PATH)
    if not cap.isOpened():
        print(f"❌ Lỗi: Không mở được file input {INPUT_PATH}")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Chuẩn bị file lưu kết quả
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (width, height))

    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break

        frame_count += 1
        if frame_count % 20 == 0: print(f"-> Đang xử lý frame {frame_count}...")

        # Convert sang RGB cho model
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect
        results_vehicle = model_vehicle(frame_rgb)
        results_helmet = model_helmet(frame_rgb)

        bikes = results_vehicle.xyxy[0].cpu().numpy()
        heads = results_helmet.xyxy[0].cpu().numpy()

        # Vẽ và lọc
        # 1. Vẽ xe máy (Màu xanh lá)
        for bike in bikes:
            bx1, by1, bx2, by2, _, _ = bike
            cv2.rectangle(frame, (int(bx1), int(by1)), (int(bx2), int(by2)), (0, 255, 0), 1)

        # 2. Xử lý mũ
        for head in heads:
            hx1, hy1, hx2, hy2, hconf, hcls = head
            label_name = model_helmet.names[int(hcls)]

            # Kiểm tra va chạm (IoU) với xe máy
            is_on_bike = False
            head_box = [hx1, hy1, hx2, hy2]

            for bike in bikes:
                bx1, by1, bx2, by2, _, _ = bike
                # Mở rộng vùng xe lên trên để bắt dính đầu người ngồi
                bike_box_extended = [bx1, by1 - (by2-by1)*0.8, bx2, by2]

                # Hàm kiểm tra giao nhau đơn giản
                x_left = max(head_box[0], bike_box_extended[0])
                y_top = max(head_box[1], bike_box_extended[1])
                x_right = min(head_box[2], bike_box_extended[2])
                y_bottom = min(head_box[3], bike_box_extended[3])

                if x_right > x_left and y_bottom > y_top: # Có giao nhau
                    is_on_bike = True
                    break

            # LOGIC QUYẾT ĐỊNH VẼ
            # Nếu là lỗi "Không mũ" -> Chỉ vẽ khi ngồi trên xe
            if label_name in ['No-Helmet', 'no-helmet', 'head']:
                if is_on_bike:
                    # Bắt lỗi: Vẽ Đỏ
                    cv2.rectangle(frame, (int(hx1), int(hy1)), (int(hx2), int(hy2)), (0, 0, 255), 2)
                    cv2.putText(frame, "KHONG DOI MU", (int(hx1), int(hy1)-10), 0, 0.6, (0, 0, 255), 2)
            else:
                # Nếu đội mũ rồi -> Vẽ Xanh Dương
                cv2.rectangle(frame, (int(hx1), int(hy1)), (int(hx2), int(hy2)), (255, 0, 0), 2)
                cv2.putText(frame, "An toan", (int(hx1), int(hy1)-10), 0, 0.6, (255, 0, 0), 2)

        out.write(frame)

    cap.release()
    out.release()
    clear_output()
    print(f"✅ HOÀN TẤT! Video đã lưu tại: {OUTPUT_PATH}")
    print("👉 Hãy tải về máy để xem.")

# Gọi hàm chạy
run_smart_detection()

✅ HOÀN TẤT! Video đã lưu tại: /content/ket_qua_da_loc.mp4
👉 Hãy tải về máy để xem.
